In [1]:
from sklearn.datasets import load_iris
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.datasets import load_wine
import scipy
import numpy as np

#### Скачиваем данные, в sklearn есть модуль datasets, он предоставляет широкий спектр наборов данных

In [2]:
data = load_iris()

#### Для оценки качества пользуемся функцией cross_val_score

In [3]:
cross_val_score(BernoulliNB(), data.data, data.target)

array([0.33333333, 0.33333333, 0.33333333])

In [4]:
cross_val_score(BernoulliNB(binarize=0.1), data.data, data.target)

array([0.39215686, 0.35294118, 0.375     ])

In [5]:
cross_val_score(BernoulliNB(binarize=1.), data.data, data.target)

array([0.66666667, 0.66666667, 0.66666667])

In [6]:
cross_val_score(BernoulliNB(binarize=1., alpha=0.1), data.data, data.target)

array([0.66666667, 0.66666667, 0.66666667])

#### Попробуем найти наилучшие значения

In [7]:
for binarize in [0., 0.1, 0.5, 1., 2, 10, 100]:
    for alpha in [0.1, 0.5, 1., 2., 10., 100.]:
        print(
            binarize, 
            alpha, 
            cross_val_score(BernoulliNB(binarize=binarize, alpha=alpha), data.data, data.target).mean()
        )

0.0 0.1 0.3333333333333333
0.0 0.5 0.3333333333333333
0.0 1.0 0.3333333333333333
0.0 2.0 0.3333333333333333
0.0 10.0 0.3333333333333333
0.0 100.0 0.3333333333333333
0.1 0.1 0.37336601307189543
0.1 0.5 0.37336601307189543
0.1 1.0 0.37336601307189543
0.1 2.0 0.37336601307189543
0.1 10.0 0.37336601307189543
0.1 100.0 0.37336601307189543
0.5 0.1 0.6597222222222222
0.5 0.5 0.6597222222222222
0.5 1.0 0.6597222222222222
0.5 2.0 0.6597222222222222
0.5 10.0 0.6597222222222222
0.5 100.0 0.6597222222222222
1.0 0.1 0.6666666666666666
1.0 0.5 0.6666666666666666
1.0 1.0 0.6666666666666666
1.0 2.0 0.6666666666666666
1.0 10.0 0.6666666666666666
1.0 100.0 0.6666666666666666
2 0.1 0.8206699346405228
2 0.5 0.8206699346405228
2 1.0 0.8206699346405228
2 2.0 0.8206699346405228
2 10.0 0.8206699346405228
2 100.0 0.8206699346405228
10 0.1 0.3333333333333333
10 0.5 0.3333333333333333
10 1.0 0.3333333333333333
10 2.0 0.3333333333333333
10 10.0 0.3333333333333333
10 100.0 0.3333333333333333
100 0.1 0.333333333333

#### В sklearn есть стандартная функция для такого поиска

In [8]:
res = GridSearchCV(BernoulliNB(), param_grid={
    'binarize': [0.,0.1, 0.5, 1., 2, 10, 100.],
    'alpha': [0.1, 0.5, 1., 2, 10., 100.]
}, cv=3).fit(data.data, data.target)

In [9]:
res.best_params_, res.best_score_

({'alpha': 0.1, 'binarize': 2}, 0.82)

In [10]:
cross_val_score(BernoulliNB(binarize=2., alpha=0.1), data.data, data.target).mean()

0.8206699346405228

#### Добавим параметры

In [11]:
res = GridSearchCV(BernoulliNB(), param_grid={
    'binarize': [0.,0.1, 0.5, 1., 2, 10, 100.],
    'alpha': [0.1, 0.5, 1., 2, 10., 100.],
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
res.best_params_, res.best_score_

({'alpha': 0.1, 'binarize': 2, 'fit_prior': False}, 0.82)

#### Иногда по сетке перебирать параметры слишком избыточное, поэтому имеет смысл использовать RandomziedSearch

In [12]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 10),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
res.best_params_, res.best_score_

({'alpha': 1.01004798124118,
  'binarize': 5.368159810138708,
  'fit_prior': False},
 0.7666666666666667)

#### Первый вариант - локализованный перебор

In [13]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(1.5, 2.5),
    'alpha': scipy.stats.uniform(0.05, 0.15),
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
res.best_params_, res.best_score_

({'alpha': 0.13632472331548187,
  'binarize': 1.7750699691560519,
  'fit_prior': False},
 0.9466666666666667)

In [14]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(1.5, 2.5),
    'alpha': scipy.stats.uniform(0.05, 0.15),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
res.best_params_, res.best_score_

({'alpha': 0.07339917805043039,
  'binarize': 1.6452090304204987,
  'fit_prior': True},
 0.92)

#### Второй вариант - больше диапозоны и больше итераций

In [15]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 10),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=1000).fit(data.data, data.target)
res.best_params_, res.best_score_

({'alpha': 6.075448519014383,
  'binarize': 1.7052412368729153,
  'fit_prior': False},
 0.9466666666666667)

#### Оценим качество MultinomialNB на той же задаче

In [16]:
cross_val_score(MultinomialNB(), data.data, data.target)

array([1.        , 0.88235294, 1.        ])

#### Подберём оптимальные параметры

In [17]:
%%time
res = GridSearchCV(MultinomialNB(), param_grid={
    'alpha': np.arange(0.1, 10.1, 0.1),
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 2.7, 'fit_prior': False} 0.9666666666666667
CPU times: user 1.13 s, sys: 4 ms, total: 1.14 s
Wall time: 1.15 s


In [18]:
%%time
res = RandomizedSearchCV(MultinomialNB(), param_distributions={
    'alpha': scipy.stats.uniform(0.1, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 6.086584841970366, 'fit_prior': False} 0.9666666666666667
CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 69.4 ms


In [19]:
%%time
res = RandomizedSearchCV(MultinomialNB(), param_distributions={
    'alpha': scipy.stats.uniform(0.1, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=200).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 6.086584841970366, 'fit_prior': False} 0.9666666666666667
CPU times: user 1.02 s, sys: 0 ns, total: 1.02 s
Wall time: 1.03 s


#### Добавим в сравнение GaussianNB

In [20]:
cross_val_score(GaussianNB(), data.data, data.target)

array([0.92156863, 0.90196078, 0.97916667])

In [21]:
cross_val_score(GaussianNB(), data.data, data.target).mean()

0.9342320261437909

#### Проверим работу методов на другом наборе данных

In [22]:
data = fetch_20newsgroups_vectorized(subset='all', remove=('headers', 'footers', 'quotes'))

In [23]:
data.data

<18846x101631 sparse matrix of type '<class 'numpy.float64'>'
	with 1769365 stored elements in Compressed Sparse Row format>

In [24]:
cross_val_score(BernoulliNB(), data.data, data.target)

array([0.46581876, 0.50334501, 0.46670914])

In [25]:
%%time
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 1),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 0.07066305219717406, 'binarize': 0.023062425041415757, 'fit_prior': False} 0.6818423007534755
CPU times: user 14 s, sys: 1.48 s, total: 15.4 s
Wall time: 15.5 s


In [26]:
cross_val_score(MultinomialNB(), data.data, data.target)

array([0.55023847, 0.54858235, 0.51258363])

In [27]:
%%time
res = RandomizedSearchCV(MultinomialNB(), param_distributions={
    'alpha': scipy.stats.uniform(0.1, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 0.10778765841014329, 'fit_prior': True} 0.6743075453677173
CPU times: user 11 s, sys: 572 ms, total: 11.6 s
Wall time: 11.7 s


In [28]:
cross_val_score(GaussianNB(), data.data, data.target)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

#### Может сложиться ощущение, что GaussianNB работает хуже, однако, когда признаки вещественные, это не так

In [29]:
data = load_wine()

In [30]:
data.data

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [31]:
cross_val_score(BernoulliNB(), data.data, data.target)

array([0.4       , 0.4       , 0.39655172])

In [32]:
%%time
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 1000),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=500).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 3.559726786512616, 'binarize': 757.8461104643691, 'fit_prior': False} 0.6966292134831461
CPU times: user 3.19 s, sys: 36 ms, total: 3.22 s
Wall time: 3.23 s


In [33]:
cross_val_score(MultinomialNB(), data.data, data.target)

array([0.71666667, 0.81666667, 0.96551724])

In [34]:
%%time
res = RandomizedSearchCV(MultinomialNB(), param_distributions={
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=500).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 3.745401188473625, 'fit_prior': False} 0.8426966292134831
CPU times: user 2.72 s, sys: 4 ms, total: 2.72 s
Wall time: 2.72 s


In [35]:
cross_val_score(GaussianNB(), data.data, data.target)

array([0.95      , 0.96666667, 0.96551724])

In [36]:
cross_val_score(GaussianNB(), data.data, data.target).mean()

0.960727969348659